In [1]:
# import tree cover data
# select the loss band
# select gain band
# sum pixels
# convert pixel into area
# print out the numbers

In [11]:
import ee
import geemap

ee.Initialize()

Map = geemap.Map()

In [18]:
raw_countries = ee.FeatureCollection('users/ccbitt23/testDHS')     # replace with some variable

countries = raw_countries.map(lambda point: point.buffer(10_000))

gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')     # also replace with variable
forest_loss = gfc2014.select('loss')

Map.addLayer(forest_loss.updateMask(forest_loss), {'palette':'FF00FF'}, name='Gain and Loss');


raw_forest_stats = forest_loss.reduceRegion(
  reducer=ee.Reducer.sum(),
  geometry=countries,
  scale=30,
  maxPixels=1e9)

forest_stats = raw_forest_stats.getInfo()
print(forest_stats)

{'loss': 482920.0784313724}
